In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear_100_0/non_linear_100_0.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/non_linear')
MODEL_RESULTS_AE_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/non_linear/model_autoencoder_AE')
MODEL_RESULTS_PGNNIV_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/non_linear/model_autoencoder_NN')


# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_AE_PATH)
create_folder(MODEL_RESULTS_PGNNIV_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/non_linear/model_autoencoder_AE
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/non_linear/model_autoencoder_NN


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_100_0/non_linear_100_0.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


### División de los datos

In [6]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = torch.Tensor(dataset['y_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train']).unsqueeze(1)
f_train = torch.tensor(dataset['f_train']).unsqueeze(1).to(torch.float32)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).to(torch.float32).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

Train dataset length: 80
Validation dataset length: 20


In [7]:
N_data_AE = len(X_train)//2
N_data_NN = len(X_train) - len(X_train)//2
prop_data_NN = 1 - N_data_AE/(N_data_NN + N_data_AE)

print("Dataset length for the autoencoder:", N_data_AE)
print("Dataset length for the PGNNIV:", N_data_NN)

X_AE, X_NN, y_AE, y_NN, K_AE, K_NN, f_AE, f_NN = train_test_split(X_train, y_train, K_train, f_train, test_size=prop_data_NN, random_state=42)

Dataset length for the autoencoder: 40
Dataset length for the PGNNIV: 40


#### Datos para el autoencoder

In [8]:
y_train_AE, y_test_AE = train_test_split(y_AE, test_size=0.2, random_state=42)

y_train_AE = TensOps(y_train_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_AE = TensOps(y_test_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

#### Datos para la PGNNIV

In [9]:
X_train_NN, X_test_NN, y_train_NN, y_test_NN, K_train_NN, K_test_NN, f_train_NN, f_test_NN = train_test_split(X_NN, y_NN, K_NN, f_NN, test_size=0.2, random_state=42)

X_train_NN = X_train_NN.to(DEVICE)
X_test_NN = X_test_NN.to(DEVICE)

y_train_NN = TensOps(y_train_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_NN = TensOps(y_test_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

K_train_NN = TensOps(K_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
K_test_NN = TensOps(K_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

f_train_NN = TensOps(f_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
f_test_NN = TensOps(f_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

## Autoencoder

In [10]:
from model import Autoencoder
from trainers.train import train_autoencoder_loop

In [11]:
n_modes = 10

autoencoder_input_shape = y_train_AE.values[0].shape
latent_space_dim = [20, 10, n_modes, 10, 20]
autoencoder_output_shape = y_train_AE.values[0].shape

In [12]:
X_train = y_train_AE.values
y_train = y_train_AE

X_test = y_test_AE.values
y_test = y_test_AE

In [13]:
autoencoder = Autoencoder(autoencoder_input_shape, latent_space_dim, autoencoder_output_shape).to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

start_epoch = 0
n_epochs = 10000
batch_size = 64
n_checkpoint = 3
new_lr = None

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from scratch.
Epoch 0, Train loss: 5.222e+01, Test loss: 9.438e+01
Epoch 100, Train loss: 3.445e+00, Test loss: 5.813e+00
Epoch 200, Train loss: 1.843e+00, Test loss: 3.632e+00
Epoch 300, Train loss: 1.722e+00, Test loss: 3.656e+00
Epoch 400, Train loss: 1.381e+00, Test loss: 3.816e+00
Epoch 500, Train loss: 1.044e+00, Test loss: 4.107e+00
Epoch 600, Train loss: 9.062e-01, Test loss: 4.107e+00
Epoch 700, Train loss: 8.623e-01, Test loss: 3.845e+00
Epoch 800, Train loss: 8.157e-01, Test loss: 3.455e+00
Epoch 900, Train loss: 6.949e-01, Test loss: 2.263e+00
Epoch 1000, Train loss: 4.248e-01, Test loss: 1.213e+00
Epoch 1100, Train loss: 3.574e-01, Test loss: 1.021e+00
Epoch 1200, Train loss: 3.341e-01, Test loss: 9.774e-01
Epoch 1300, Train loss: 3.204e-01, Test loss: 9.515e-01
Epoch 1400, Train loss: 3.106e-01, Test loss: 9.304e-01
Epoch 1500, Train loss: 3.000e-01, Test loss: 9.048e-01
Epoch 1600, Train loss: 2.916e-01, Test loss: 8.802e-01
Epoch 1700, Train loss: 2.84

In [15]:
start_epoch = 9999
n_epochs = 20000
batch_size = 64
n_checkpoint = 3
new_lr = 1e-4

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from a checkpoint. Epoch 9999.
Epoch 10000, Train loss: 6.665e-02, Test loss: 3.442e-01
Epoch 10100, Train loss: 6.657e-02, Test loss: 3.438e-01
Epoch 10200, Train loss: 6.650e-02, Test loss: 3.436e-01
Epoch 10300, Train loss: 6.644e-02, Test loss: 3.434e-01
Epoch 10400, Train loss: 6.637e-02, Test loss: 3.432e-01
Epoch 10500, Train loss: 6.629e-02, Test loss: 3.429e-01
Epoch 10600, Train loss: 6.621e-02, Test loss: 3.426e-01
Epoch 10700, Train loss: 6.612e-02, Test loss: 3.423e-01
Epoch 10800, Train loss: 6.602e-02, Test loss: 3.419e-01
Epoch 10900, Train loss: 6.591e-02, Test loss: 3.415e-01
Epoch 11000, Train loss: 6.579e-02, Test loss: 3.411e-01
Epoch 11100, Train loss: 6.566e-02, Test loss: 3.406e-01
Epoch 11200, Train loss: 6.551e-02, Test loss: 3.400e-01
Epoch 11300, Train loss: 6.535e-02, Test loss: 3.394e-01
Epoch 11400, Train loss: 6.517e-02, Test loss: 3.387e-01
Epoch 11500, Train loss: 6.496e-02, Test loss: 3.379e-01
Epoch 11600, Train loss: 6.474e-02, Tes

## PGNNIV

In [16]:
from vecopsciml.operators.zero_order import Mx, My
from model.ae_nonlinear_model import AutoencoderNonlinearModel

In [17]:
# Predictive network architecture
input_shape = X_train_NN[0].shape
predictive_layers = [20, 10, n_modes]
predictive_output = y_train_NN.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train_NN)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train_NN)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [18]:
pretrained_decoder = autoencoder.decoder

for param in pretrained_decoder.parameters():
    param.requires_grad = False

for name, param in pretrained_decoder.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

hidden1_layer.weight: requires_grad=False
hidden1_layer.bias: requires_grad=False
hidden2_layer.weight: requires_grad=False
hidden2_layer.bias: requires_grad=False
output_layer.weight: requires_grad=False
output_layer.bias: requires_grad=False


In [21]:
model = AutoencoderNonlinearModel(input_shape, predictive_layers, pretrained_decoder, predictive_output, explanatory_input,
                                   explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 20000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 8.435e+10, Test loss: 1.184e+11, MSE(e): 8.433e+00, MSE(pi1): 1.116e+00, MSE(pi2): 3.999e+00, MSE(pi3): 1.032e-01
Epoch 100, Train loss: 1.759e+10, Test loss: 1.596e+10, MSE(e): 1.757e+00, MSE(pi1): 1.035e+00, MSE(pi2): 8.104e-01, MSE(pi3): 1.021e-01
Epoch 200, Train loss: 1.437e+10, Test loss: 1.273e+10, MSE(e): 1.435e+00, MSE(pi1): 9.642e-01, MSE(pi2): 6.718e-01, MSE(pi3): 9.879e-02
Epoch 300, Train loss: 7.485e+09, Test loss: 8.098e+09, MSE(e): 7.471e-01, MSE(pi1): 6.569e-01, MSE(pi2): 3.821e-01, MSE(pi3): 7.673e-02
Epoch 400, Train loss: 4.257e+09, Test loss: 8.362e+09, MSE(e): 4.247e-01, MSE(pi1): 4.899e-01, MSE(pi2): 2.433e-01, MSE(pi3): 4.563e-02
Epoch 500, Train loss: 3.518e+09, Test loss: 6.709e+09, MSE(e): 3.509e-01, MSE(pi1): 5.031e-01, MSE(pi2): 2.098e-01, MSE(pi3): 4.386e-02
Epoch 600, Train loss: 3.018e+09, Test loss: 5.471e+09, MSE(e): 3.009e-01, MSE(pi1): 5.053e-01, MSE(pi2): 1.837e-01, MSE(pi3): 4.059e-02
Epoch 700, 

In [23]:
# Parametros de entrenamiento
start_epoch = 18000
n_epochs = 100000

batch_size = 64 
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 18000.
Epoch 18000, Train loss: 1.369e+07, Test loss: 8.295e+07, MSE(e): 1.280e-03, MSE(pi1): 7.035e-02, MSE(pi2): 1.336e-03, MSE(pi3): 1.816e-03
Epoch 18100, Train loss: 1.309e+07, Test loss: 7.856e+07, MSE(e): 1.220e-03, MSE(pi1): 7.098e-02, MSE(pi2): 1.331e-03, MSE(pi3): 1.777e-03
Epoch 18200, Train loss: 1.309e+07, Test loss: 7.854e+07, MSE(e): 1.220e-03, MSE(pi1): 7.098e-02, MSE(pi2): 1.331e-03, MSE(pi3): 1.777e-03
Epoch 18300, Train loss: 1.309e+07, Test loss: 7.853e+07, MSE(e): 1.220e-03, MSE(pi1): 7.098e-02, MSE(pi2): 1.331e-03, MSE(pi3): 1.777e-03
Epoch 18400, Train loss: 1.309e+07, Test loss: 7.853e+07, MSE(e): 1.220e-03, MSE(pi1): 7.097e-02, MSE(pi2): 1.331e-03, MSE(pi3): 1.777e-03
Epoch 18500, Train loss: 1.309e+07, Test loss: 7.852e+07, MSE(e): 1.220e-03, MSE(pi1): 7.097e-02, MSE(pi2): 1.331e-03, MSE(pi3): 1.777e-03
Epoch 18600, Train loss: 1.309e+07, Test loss: 7.851e+07, MSE(e): 1.220e-03, MSE(pi1): 7.097e-02, MSE(pi2): 1.331e-0